This is an example run through to show how the match_audio_to_video workflow goes


Author : Thejasvi Beleyur, August 2019,
         Acoustic and Functional Ecology Group, Max Planck Institute for Ornithology, Seewiesen

In [1]:
import sys 
sys.path.append('../bin/') # include the modules in the outer folder in the search path
import matplotlib.pyplot as plt 
import pandas as pd
import numpy as np 
import datetime as dt
import os


In [2]:
%matplotlib notebook

### Getting your borders right:
For reliable and quick light intensity tracking and timestamp reading it is best to provide a narrow region around the focal area. In my experience I've seen that the timestamp reading can get a bit wonky if the whole image is provided!

In [3]:
# Get the led and timestamp borders 
%run ../bin/browse_through_video.py -v '/media/tbeleyur/PEN_DRIVe/AV_synchronisation_experiment_4July2019/video/DVRecorder_03_20190704_16.49.45-16.56.42[R][@da37][0].avi'

TypeError: Image data cannot be converted to float

So - for the example video the led_border entry should be (418.09, 792.16, 492.46, 246.62) *your points may be different than mine!!* - don't bother with too many decimal places! - repeat this once more to get the timestamp_border - and do this for every video file you have! 
The points are chosen by left click. If you want to delete an accidental click then use the right click. 
If you think the 4 points you chose still aren't right - no worries you can press the choose border button once more and redo the whole process. 

You could redo the border selection any time by keeping the frame-by-frame display open!

The border output you need to put into the annotations file is always printed out. 


### Running the video file level data production : tracking the light intensity and timestamps frame-by-frame
After entering your annotations, now run the  routine that will measure the light intensity in each frame and read the timestamp. 

In [4]:
from generate_data_from_video import generate_videodata_from_videofiles

annotations_df = pd.read_csv('5am-6am.csv')
# Since it's a small video with only 1370 frames - we'll run the whole thing! This could take a couple of minutes


IOError: [Errno 2] File 5am-6am.csv does not exist: '5am-6am.csv'

The step above should have given a .csv file with the following template : *videosync_file path here_.csv*. You will notice the 'timestamp_verified' columns is blank. This is because the effectiveness of the OCR can vary a lot. Copy the 'timestamp' column into timestamp_verified and correct the entries wherever they look weird. It may be useful especially to check along transition points ie. where the second changes or the date changes - sometimes the change in the numbers are not detected. 

This is also the point at which you can run *browse_through_video* and run through the videos keeping the interface on inspect video mode. For instance you might see a misread at frame 186 (indexing starts from 1 in the package!), and some others at 16:26:25. 


In [ ]:
df = pd.read_csv('videosync_DVRecorder_03_20190704_16.49.45-16.56.42[R][@da37][0].avi_.csv')

In [ ]:
timestamps, counts = np.unique(df['timestamp'], return_counts=True)

In [ ]:
display_framerate = pd.DataFrame(data={'timestamp':timestamps,'frames_counted':counts})

In [ ]:
display_framerate

In [ ]:
plt.figure()
plt.plot(df['led_intensity'])

### Resampling the led signal and checking for dropped frames :
The *videosync* file is raw data. Depending on your camera+DVR system there could be dropped frames or the framerate may vary over time. These issues need to be detected and corrected for at the level of each annotation. 

The output of this step is a series of *commonfps* files.

In [3]:
from process_video_annotations import video_sync_over_annotation_block

annotations = pd.read_csv('eg_annotations.csv')
whole_video_data = pd.read_csv('videosync_DVRecorder_03_20190704_16.49.45-16.56.42[R][@da37][0].avi_.csv')

kwargs = {'timestamp_pattern': '%Y-%m-%d %H:%M:%S'}
kwargs['min_fps']= 20 # Hz
kwargs['min_durn'] = 20.0 # seconds 
kwargs['common_fps'] = 25 # Hz
kwargs['output_folder'] = './'
success = annotations.apply(video_sync_over_annotation_block,1, video_sync_data=whole_video_data , 
                                 **kwargs)
print(success)


/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2223: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2225: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Y[sl] = X[sl]
/home/tbeleyur/anaconda2/envs/horseshoebat/lib/python2.7/site-packages/scipy/signal/signaltools.py:2233: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` 

Saved annotation :202
Saved annotation :555
0    True
1    True
dtype: bool


In [4]:
annotations.head()

,video_path,annotation_id,start_timestamp,start_framenumber,end_timestamp,end_framenumber,timestamp_border,led_border
0,./vlc-record-2019-08-19-16h32m09s-DVRecorder_0...,202,2019-07-04 16:54:00,1,2019-07-04 16:54:02,20,"(587.0846069412526, 57.53531668779959, 111.773...","(418.5748905322298, 796.1543636674143, 497.574..."
1,.vlc-record-2019-08-19-16h32m09s-DVRecorder_03...,555,2019-07-04 16:51:40,1,2019-07-04 16:51:55,15,"(587.2505532565692, 51.974969803362015, 112.16...","(416.26279113581603, 796.2260524351722, 496.60..."


In [3]:
def write_match_report(all_match_cc, all_files):
    df = pd.DataFrame(data={'file_name':all_files, 'match_cc':all_match_cc})
    now = dt.datetime.now().strftime('%Y-%m-%d %H-%M-%S')
    df.to_csv('AV-match-report_'+now+'.csv')
    

In [5]:
### Finding the matching audio snippet !! 
import glob 
import soundfile as sf
from audio_for_videoannotation import match_video_sync_to_audio

#all_commonfps = glob.glob('common_fps_video_sync*') # get all the relevant common_fps_sync files
all_commonfps = glob.glob('../experimental_testdata/horseshoebat_data/common_fps/common_fps*')
audio_folder = '../experimental_testdata/horseshoebat_data/audio/' # the current folder
audiosync_folder = '../experimental_testdata/horseshoebat_data/sync_audio/'
audioannotation_folder = '../experimental_testdata/horseshoebat_data/annotation_audio/'
fs = 250000 # change according to the recording sampling rate!! 
# generate the 

In [6]:
all_ccs = []
files_to_run = sorted(all_commonfps)[1:2]
for somenumber, each_commonfps in enumerate(files_to_run):
    print(each_commonfps)
    video_sync = pd.read_csv(each_commonfps)
    best_audio, syncblock_audio, crosscoef = match_video_sync_to_audio(video_sync, audio_folder, audio_fileformat='*.WAV',
                                           audio_sync_spikey=False)
    all_ccs.append(crosscoef)
    fname  = os.path.split(each_commonfps)[-1]
    annotation_id = '-'.join(os.path.split(fname)[-1].split('_')[-2:])
                                                                 
    try:
        sf.write(audiosync_folder+'matching_sync_'+str(somenumber)+'.WAV', syncblock_audio,fs)
        sf.write(audioannotation_folder+'matching_annotaudio_'+str(somenumber)+'.WAV', best_audio,fs)
    except:
        print('Could not save ', each_commonfps)

#write_match_report(all_ccs, sorted(all_commonfps))



  0%|          | 0/70 [00:00<?, ?it/s]

../experimental_testdata/horseshoebat_data/common_fps/common_fps_video_sync56_10.csv
('video_fps obtained is :', 25)
Did not find user-provided sample rate - getting it from first file that matches format!
('sampling rate is : ', 250000)
.....finding best audio segment.....


Unable to get proper audio match for video segment!


MemoryError: 